### Building an image classification model using very little data  

Based on the tutorial by Francois Chollet @fchollet https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and the workbook by Guillaume Dominici https://github.com/gggdominici/keras-workshop

This tutorial presents several ways to build an image classifier using keras from just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:  

- training a small network from scratch (as a baseline)  
- using the bottleneck features of a pre-trained network  
- fine-tuning the top layers of a pre-trained network  
  
This will lead us to cover the following Keras features:   
  
- fit_generator for training Keras a model using Python data generators  
- ImageDataGenerator for real-time data augmentation  
- layer freezing and model fine-tuning  
- ...and more.  


## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data  
All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```
Note : for this example we only consider 2x1000 training images and 2x400 testing images among the 2x12500 available.

The github repo includes about 1500 images for this model. The original Kaggle dataset is much larger. The purpose of this demo is to show how you can build models with smaller size datasets. You should be able to improve this model by using more data.

### Data loading

In [ ]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

In [5]:
import glob,random,shutil
from sklearn.utils import shuffle
cats = glob.glob("data0/train/cats/*.jpg")
dogs = glob.glob("data0/train/dogs/*.jpg")
# cats = shuffle(cats)
# dogs = shuffle(dogs)

for i in range(2048):
    shutil.copy(cats[i], "data/train/cats/cat%04d.jpg"%i)
    shutil.copy(dogs[i], "data/train/dogs/dog%04d.jpg"%i)
for i in range(416):
    shutil.copy(cats[i], "data/validation/cats/cat%04d.jpg"%i)
    shutil.copy(dogs[i], "data/validation/dogs/dog%04d.jpg"%i)
print("done")

done


In [6]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D
from keras import optimizers

In [7]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [8]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 4096 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


## Small Conv Net

### Model architecture definition

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Training

In [11]:
nb_epoch = 30
nb_train_samples = 4096
nb_validation_samples = 832

In [12]:
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

/home/jidou/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=30, validation_data=<keras.pre..., steps_per_epoch=128, validation_steps=832)`
  '` call to the Keras 2 API: ' + signature)


Epoch 1/30
128/128 [==============================] - 53s - loss: 0.7121 - acc: 0.5447 - val_loss: 0.6806 - val_acc: 0.5493

In [13]:
model.save_weights('models/basic_cnn_20_epochs.h5')
print("model saved")

model saved


In [ ]:
# model.load_weights('models/basic_cnn_20_epochs.h5')
# print("model loaded")

If your model successfully runs at one epoch, go back and it for 30 epochs by changing nb_epoch above.  I was able to get to an val_acc of 0.71 at 30 epochs.
A copy of a pretrained network is available in the pretrained folder.

### Evaluating on validation set

Computing loss and accuracy :

In [14]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.12423450714693619, 0.95673076923076927]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_no_dataaugmentation.png)

**After ~10 epochs the neural network reach ~70% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs**

## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of datas.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

### VGG16 + small MLP
![VGG16 + Dense layers Schema](pictures/vgg16_original.png)

#### VGG16 model architecture definition

In [15]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Conv2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(128, (3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#### Loading VGG16 weights
This part is a bit complicated because the structure of our model is not exactly the same as the one used when training weights.  
Otherwise, we would use the `model.load_weights()` method.

*Note : the VGG16 weights file (~500MB) is not included in this repository. You can download from here :  
https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3*

In [16]:
import h5py
f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]
    print("{} -- {}".format(layer.__class__.__name__, f['layer_{}'.format(k)]))
    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D','Conv1D','Conv2D','Conv3D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    layer.set_weights(weights)
f.close()
print("load vgg16 model successed")

ZeroPadding2D -- <HDF5 group "/layer_0" (0 members)>
Conv2D -- <HDF5 group "/layer_1" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_2" (0 members)>
Conv2D -- <HDF5 group "/layer_3" (2 members)>
MaxPooling2D -- <HDF5 group "/layer_4" (0 members)>
ZeroPadding2D -- <HDF5 group "/layer_5" (0 members)>
Conv2D -- <HDF5 group "/layer_6" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_7" (0 members)>
Conv2D -- <HDF5 group "/layer_8" (2 members)>
MaxPooling2D -- <HDF5 group "/layer_9" (0 members)>
ZeroPadding2D -- <HDF5 group "/layer_10" (0 members)>
Conv2D -- <HDF5 group "/layer_11" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_12" (0 members)>
Conv2D -- <HDF5 group "/layer_13" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_14" (0 members)>
Conv2D -- <HDF5 group "/layer_15" (2 members)>
MaxPooling2D -- <HDF5 group "/layer_16" (0 members)>
ZeroPadding2D -- <HDF5 group "/layer_17" (0 members)>
Conv2D -- <HDF5 group "/layer_18" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_19" (0

### Using the VGG16 model to process samples

In [17]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

Found 4096 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


This is a long process, so we save the output of the VGG16 once and for all.  

In [18]:
print(nb_train_samples)
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, nb_train_samples/32)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
print("Done {}".format(bottleneck_features_train.shape))

4096
Done (4096, 4, 4, 512)


In [19]:
print(nb_validation_samples)
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, nb_validation_samples/32)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
print("Done {}".format(bottleneck_features_validation.shape))

832
Done (832, 4, 4, 512)


Now we can load it...

In [20]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
print(train_data.shape)
print(train_labels.shape)

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
print(validation_data.shape)
print(validation_labels.shape)

(4096, 4, 4, 512)
(4096,)
(832, 4, 4, 512)
(832,)


And define and train the custom fully connected neural network :

In [21]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
nb_epoch=40
model_top.fit(train_data, train_labels,
          epochs=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels))

Train on 4096 samples, validate on 832 samples
Epoch 1/40
4096/4096 [==============================] - 0s - loss: 0.7400 - acc: 0.7200 - val_loss: 0.3576 - val_acc: 0.8305
Epoch 2/40
4096/4096 [==========================

The training process of this small neural network is very fast : ~2s per epoch

In [23]:
model_top.save_weights('models/bottleneck_40_epochs.h5')

### Bottleneck model evaluation

In [ ]:
#model_top.load_weights('models/with-bottleneck/1000-samples--100-epochs.h5')
#model_top.load_weights('/notebook/Data1/Code/keras-workshop/models/with-bottleneck/1000-samples--100-epochs.h5')

Loss and accuracy :

In [24]:
model_top.evaluate(validation_data, validation_labels)

 32/832 [>.............................] - ETA: 0s

[0.00092447972383524757, 1.0]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_with_bottleneck.png)

**We reached a 90% accuracy on the validation after ~1m of training (~20 epochs) and 8% of the samples originally available on the Kaggle competition !**

In [ ]:
##Fine-tuning the top layers of a a pre-trained network

Start by instantiating the VGG base and loading its weights.

In [25]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Conv2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(128, (3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

print("Done")

Done


In [26]:
import h5py
f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]
    print("{} -- {}".format(layer.__class__.__name__, f['layer_{}'.format(k)]))
    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D','Conv1D','Conv2D','Conv3D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    layer.set_weights(weights)
f.close()

ZeroPadding2D -- <HDF5 group "/layer_0" (0 members)>
Conv2D -- <HDF5 group "/layer_1" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_2" (0 members)>
Conv2D -- <HDF5 group "/layer_3" (2 members)>
MaxPooling2D -- <HDF5 group "/layer_4" (0 members)>
ZeroPadding2D -- <HDF5 group "/layer_5" (0 members)>
Conv2D -- <HDF5 group "/layer_6" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_7" (0 members)>
Conv2D -- <HDF5 group "/layer_8" (2 members)>
MaxPooling2D -- <HDF5 group "/layer_9" (0 members)>
ZeroPadding2D -- <HDF5 group "/layer_10" (0 members)>
Conv2D -- <HDF5 group "/layer_11" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_12" (0 members)>
Conv2D -- <HDF5 group "/layer_13" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_14" (0 members)>
Conv2D -- <HDF5 group "/layer_15" (2 members)>
MaxPooling2D -- <HDF5 group "/layer_16" (0 members)>
ZeroPadding2D -- <HDF5 group "/layer_17" (0 members)>
Conv2D -- <HDF5 group "/layer_18" (2 members)>
ZeroPadding2D -- <HDF5 group "/layer_19" (0

Build a classifier model to put on top of the convolutional model. For the fine tuning, we start with a fully trained-classifer. We will use the weights from the earlier model. And then we will add this model on top of the convolutional base.

In [27]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights('models/bottleneck_40_epochs.h5')

model_vgg.add(top_model)

For fine turning, we only want to train a few layers.  This line will set the first 25 layers (up to the conv block) to non-trainable.

In [28]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False

In [29]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model_vgg.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [30]:
# prepare data augmentation configuration  . . . do we need this?
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 4096 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


In [31]:
# fine-tune the model
model_vgg.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

/home/jidou/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=128, epochs=40, validation_steps=832)`
  '` call to the Keras 2 API: ' + signature)


Epoch 1/40
128/128 [==============================] - 69s - loss: 0.3986 - acc: 0.8474 - val_loss: 0.1019 - val_acc: 0.9736

In [32]:
model_vgg.save_weights('models/finetuning_20epochs_vgg.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [33]:
model_vgg.evaluate_generator(validation_generator, nb_validation_samples)

[0.0064405485844382876, 1.0]

In [34]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.1242345141318555, 0.95673076923076927]

In [35]:
model_top.evaluate(validation_data, validation_labels)

 32/832 [>.............................] - ETA: 0s

[0.00092447972383524757, 1.0]